In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import torch
import lightning.pytorch as pl

from mnist_datamodule import MNISTDataModule
from variational_auto_encoders_module import VAE

In [2]:
import torchvision.transforms as transforms
from torchvision import datasets


class MNISTToRGB:
    def __call__(self, img):
        img = torch.cat([img, img, img], dim=0)  # Duplicate the single channel into three channels
        return img
train_transform = transforms.Compose(
            [
                transforms.Resize((32, 32)),
                transforms.ToTensor(),
                MNISTToRGB(),  # Convert to 3 channels
                transforms.Normalize((0.1307, 0.1307, 0.1307), (0.3081, 0.3081, 0.3081)),
            ]
        )
datamodule = MNISTDataModule(val_batch_size=1,train_augments=train_transform)

In [3]:
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch import Trainer

# Optional: Logging with Weights & Biases
logger = [WandbLogger(project="MNIST_VAETRIAL"),CSVLogger(save_dir="csv_logs")]
model = VAE()
trainer = Trainer( max_epochs=1, accelerator="cpu",logger=logger)
trainer.fit(model=model, datamodule=datamodule)

wandb: Currently logged in as: padmanabh (padmanabh275). Use `wandb login --relogin` to force relogin


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type   | Params
--------------------------------
0 | net  | VAENet | 20.9 M
--------------------------------
20.9 M    Trainable params
0         Non-trainable params
20.9 M    Total params
83.751    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

TypeError: __call__() got an unexpected keyword argument 'image'

In [ ]:
trainer.save_checkpoint("mnist_model.ckpt")

In [ ]:
vae = VAE.load_from_checkpoint("mnist_model.ckpt")

In [ ]:
from utils import plot_vae_examples
test_loader = datamodule.val_dataloader()
classes = datamodule.class_names
mean = (0.1407, 0.1407, 0.1407)
std = (0.4081, 0.4081, 0.4081)
vae.to("cpu")
plot_vae_examples(test_loader,classes, vae.net, mean, std)